In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


# **銀行解約データセットを使用した二項分類**
Binary Classification with a Bank Churn Dataset(https://www.kaggle.com/competitions/playground-series-s4e1/code?competitionId=65711&sortBy=voteCount)

# 参考資料
**書籍**


**Kaggle**
- [[PlayGround S4E1] 📊 EDA + 🤖 Modeling [XGBoost]](https://www.kaggle.com/code/akhiljethwa/playground-s4e1-eda-modeling-xgboost)
- [🏛️Binary Classification💸 | Bank Churn💰| EDA📈](https://www.kaggle.com/code/tarundirector/binary-classification-bank-churn-eda)


**自分で作成したファイル**
- 肝硬変の転帰の多クラス予測

**その他**


In [ ]:
import numpy as np
import pandas as pd

FILE_PATH = '/kaggle/input/playground-series-s4e1/'
CREATE_DATA = '/kaggle/input/my-binary-classification-with-a-bank-churn-dataset'


# 確認 (data_import.py)
def file_to_xy(filename):
    data = pd.read_csv(filename, index_col=0)
    print(f'読み込み完了 {filename}')
    
    target_col = 'Exited'
    
    train = data[:165034].reset_index(drop=True)
    test = data[165034:].reset_index(drop=True).drop(target_col, axis=1) # 目的変数を指定する
    # 目的変数と説明変数に分割
    X = train.drop([target_col], axis=1) # 目的変数を指定する
    y = train[target_col].values # 目的変数を指定する

    return data,test,train,X,y

filename = 'data.csv'
data,test,train,X,y = file_to_xy(CREATE_DATA + '/' + filename)

# 機械学習モデルの構築・学習・予測

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

sample_sub = pd.read_csv(FILE_PATH + "sample_submission.csv")

In [4]:
# 混同行列
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 機械学習モデルのフォーマットを設定
seed = 42

xgb = XGBClassifier(random_state=seed)
lgb = LGBMClassifier(random_state=seed)
cb = CatBoostClassifier(random_state=seed, verbose=0)

submit = False

## ハイパーパラメータ(optuna)

### Catboost

In [5]:
if submit==False:

    import optuna
    from catboost import CatBoostClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score

    # Assuming 'X' is your feature matrix and 'y' is your target variable
    X = X.copy()
    y = y.copy()

    def objective(trial):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        params = {
            'iterations': trial.suggest_int('iterations', 200, 1000),
            'depth': trial.suggest_int('depth', 3, 10),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 20),
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.2, log=True),
            'random_state': 42,
            'verbose': 0,
            'eval_metric': 'AUC',
        }

        model = CatBoostClassifier(**params)

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50)

        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)

        return auc

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    trial = study.best_trial

    print('Value: ', trial.value)
    print('Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

[I 2024-01-30 00:59:53,352] A new study created in memory with name: no-name-f968bd10-312b-4e6a-b1d5-46dfc747ee8c
[I 2024-01-30 01:00:01,446] Trial 0 finished with value: 0.8875184652538644 and parameters: {'iterations': 286, 'depth': 3, 'min_data_in_leaf': 6, 'learning_rate': 0.027783838816486113}. Best is trial 0 with value: 0.8875184652538644.
[I 2024-01-30 01:00:08,382] Trial 1 finished with value: 0.8710008727774776 and parameters: {'iterations': 534, 'depth': 4, 'min_data_in_leaf': 18, 'learning_rate': 0.0003795921372782072}. Best is trial 0 with value: 0.8875184652538644.
[I 2024-01-30 01:00:21,327] Trial 2 finished with value: 0.8811700853118737 and parameters: {'iterations': 598, 'depth': 3, 'min_data_in_leaf': 18, 'learning_rate': 0.004224159534024791}. Best is trial 0 with value: 0.8875184652538644.
[I 2024-01-30 01:00:38,160] Trial 3 finished with value: 0.8911998984942243 and parameters: {'iterations': 663, 'depth': 7, 'min_data_in_leaf': 18, 'learning_rate': 0.03505462349

Number of finished trials:  100
Best trial:
Value:  0.8914165640957206
Params: 
    iterations: 936
    depth: 5
    min_data_in_leaf: 2
    learning_rate: 0.09339658816165013


### XGBClassifier

In [6]:
if submit == False:
    import optuna
    from xgboost import XGBClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score

    # Assuming 'X' is your feature matrix and 'y' is your target variable
    X = X.copy()
    y = y.copy()

    def objective(trial):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        params = {
            'max_depth': trial.suggest_int('max_depth', 5, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 150, 1000),
            'subsample': trial.suggest_float('subsample', 0.01, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
            'random_state': trial.suggest_categorical('random_state', [42]),
            'tree_method': 'hist',  # Use GPU for training
            'device': 'cuda',
            'njobs': -1,
            'eval_metric': 'auc',  # Evaluation metric
            'verbosity': 2,  # Set verbosity to 0 for less output
        }

        model = XGBClassifier(**params)

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)

        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)

        return auc

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    trial = study.best_trial

    print('Value: ', trial.value)
    print('Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

[I 2024-01-30 01:27:09,947] A new study created in memory with name: no-name-9fc8e207-4089-4528-ab90-c5314859d5ec
[I 2024-01-30 01:27:12,347] Trial 0 finished with value: 0.8802255357669333 and parameters: {'max_depth': 10, 'min_child_weight': 7, 'learning_rate': 0.5684095377562918, 'n_estimators': 601, 'subsample': 0.2753832202192548, 'colsample_bytree': 0.056427800837918006, 'random_state': 42}. Best is trial 0 with value: 0.8802255357669333.
[I 2024-01-30 01:27:15,388] Trial 1 finished with value: 0.8908004568201427 and parameters: {'max_depth': 7, 'min_child_weight': 3, 'learning_rate': 0.02941485754642511, 'n_estimators': 409, 'subsample': 0.6059567838169045, 'colsample_bytree': 0.8848453638039051, 'random_state': 42}. Best is trial 1 with value: 0.8908004568201427.
[I 2024-01-30 01:27:17,343] Trial 2 finished with value: 0.8881485025304143 and parameters: {'max_depth': 10, 'min_child_weight': 8, 'learning_rate': 0.16461469321049463, 'n_estimators': 441, 'subsample': 0.45321753562

Number of finished trials:  100
Best trial:
Value:  0.8917098391835474
Params: 
    max_depth: 5
    min_child_weight: 5
    learning_rate: 0.040082551237260594
    n_estimators: 790
    subsample: 0.7867296084411586
    colsample_bytree: 0.5387242089823288
    random_state: 42


### LGBMClassifier

In [7]:
if submit == False:
    import optuna
    import lightgbm as lgb
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score

    # Assuming 'X' is your feature matrix and 'y' is your target variable
    X = X.copy()
    y = y.copy()

    def objective(trial):
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        params = {
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'metric': 'auc',
            'max_depth': trial.suggest_int('max_depth', 5, 10),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 150, 1000),
            'subsample': trial.suggest_float('subsample', 0.1, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
#             'num_leaves': 
            'random_state': 42,
        }

        model = lgb.LGBMClassifier(**params)

        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)

        y_pred = model.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)

        return auc

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    print('Number of finished trials: ', len(study.trials))
    print('Best trial:')
    trial = study.best_trial

    print('Value: ', trial.value)
    print('Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

[I 2024-01-30 01:30:59,457] A new study created in memory with name: no-name-ed247cf7-6046-41be-891f-2a0605752060
[I 2024-01-30 01:31:01,313] Trial 0 finished with value: 0.8839152447745112 and parameters: {'max_depth': 10, 'min_child_samples': 17, 'learning_rate': 0.956346808825856, 'n_estimators': 627, 'subsample': 0.8136790500353968, 'colsample_bytree': 0.6816346669470251, 'reg_alpha': 0.8304068288132298, 'reg_lambda': 0.30227751557883165}. Best is trial 0 with value: 0.8839152447745112.
[I 2024-01-30 01:31:03,245] Trial 1 finished with value: 0.8882974139096689 and parameters: {'max_depth': 6, 'min_child_samples': 14, 'learning_rate': 0.6949087926255573, 'n_estimators': 624, 'subsample': 0.7771337499481712, 'colsample_bytree': 0.7321621703807396, 'reg_alpha': 0.38576685289234236, 'reg_lambda': 0.07383788598231245}. Best is trial 1 with value: 0.8882974139096689.
[I 2024-01-30 01:31:05,129] Trial 2 finished with value: 0.8872763431826829 and parameters: {'max_depth': 7, 'min_child_s

Number of finished trials:  100
Best trial:
Value:  0.8918460237077137
Params: 
    max_depth: 5
    min_child_samples: 9
    learning_rate: 0.020567479623411933
    n_estimators: 974
    subsample: 0.21658773194907643
    colsample_bytree: 0.4746101648416433
    reg_alpha: 0.6252030731003401
    reg_lambda: 0.6872606202657469


### パラメータ

In [8]:
lgb_params = {'max_depth': 7,
    'min_child_samples': 1,
    'learning_rate': 0.04163336702197924,
    'n_estimators': 982,
    'subsample': 0.10132257615206398,
    'colsample_bytree': 0.3686160004388141,
    'reg_alpha': 0.6838448061147125,
    'reg_lambda': 0.8044499417370345}

xgb_params = {'max_depth': 5,
    'min_child_weight': 7,
    'learning_rate': 0.07500811721987605,
    'n_estimators': 521,
    'subsample': 0.9536911064926733,
    'colsample_bytree': 0.3557155199600278,
    'random_state': 42,
    'njobs': -1}

cat_params = {'iterations': 984,
    'depth': 6,
    'min_data_in_leaf': 3,
    'learning_rate': 0.04028812699101321}

In [9]:
lgb_model = LGBMClassifier(**lgb_params)
xgb_model = XGBClassifier(**xgb_params)
cat_model = CatBoostClassifier(**cat_params)